In [ ]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = [8, 3]
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels

import scipy
from scipy.stats import pearsonr

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

### Part 1: Exploratory Analysis 
Questions
1. What is wrong with the data and plot above? How can we fix this?
2. How can we make the index more time aware?
3. How strongly do these measurements correlate contemporaneously? What about with a time lag?
4. consider adding a seasonal term for 12 periods for the model fit above. Does this improve the fit of the model?

In [ ]:
# Question 1
df = pd.read_csv("global_annual_temps.csv")
df.head()
df.Mean[:100].plot()

In [ ]:
# Question 2
df.index = pd.to_datetime(df.index)
type(df.index)

In [ ]:
# Question 3
plt.scatter(df['1880':'1900'][['GCAG']], df['1880':'1900'][['GISTEMP']])
plt.scatter(df['1880':'1899'][['GCAG']], df['1881':'1900'][['GISTEMP']])
df['1880':'1899'][['GCAG']].head()
df['1881':'1900'][['GISTEMP']].head()

min(df.index)
max(df.index)

### Part 2: Modeling

In [ ]:
train = df['1960':]
# smooth trend model without seasonal or cyclical components
model = {
    'level': 'smooth trend', 'cycle': False, 'seasonal': None, 
}

# taking from statsmodel api: UnobservedComponents
gcag_mod = sm.tsa.UnobservedComponents(train['GCAG'], **model)
gcag_res = gcag_mod.fit()

In [ ]:
# plotting
fig = gcag_res.plot_components(legend_loc='lower right', figsize=(15, 9));

# Perform rolling prediction and multistep forecast
num_steps = 20
predict_res = gcag_res.get_prediction(dynamic=train['GCAG'].shape[0] - num_steps)

predict = predict_res.predicted_mean
ci = predict_res.conf_int()

In [ ]:
plt.plot(predict)
plt.scatter(train['GCAG'], predict)

In [ ]:
fig, ax = plt.subplots()
# Plot the results
ax.plot(train['GCAG'], 'k.', label='Observations');
ax.plot(train.index[:-num_steps], predict[:-num_steps], label='One-step-ahead Prediction');

ax.plot(train.index[-num_steps:], predict[-num_steps:], 'r', label='Multistep Prediction');
ax.plot(train.index[-num_steps:], ci.iloc[-num_steps:], 'k--');

In [ ]:
# Question 4 **** 
seasonal_model = {
    'level': 'local linear trend',
    'seasonal': 12
}
mod = sm.tsa.UnobservedComponents(train['GCAG'], **seasonal_model)
res = mod.fit(method='powell', disp=False)

In [ ]:
fig = res.plot_components(legend_loc='lower right', figsize=(15, 9));

In [ ]:
# Comparing the two models:
np.mean(np.abs(gcag_res.predict() - train['GCAG']))

In [ ]:
np.mean(np.abs(res.predict() - train['GCAG']))